# KGTK Browser Cache Setup

This note book will create the SQLite DB Cache and the required indices for KGTK Browser.

The required input parameters are:
- input_path: Path where the following files should be present
    1. labels.en.tsv.gz
    2. aliases.en.tsv.gz
    3. descriptions.en.tsv.gz
    4. claims.tsv.gz
    5. metadata.property.datatypes.tsv.gz
    6. qualifiers.tsv.gz
    7. metadata.pagerank.undirected.tsv.gz
    8. class-visualization.edge.tsv.gz **# optional, required for class visualization graph**
    9. class-visualization.node.tsv.gz **# optional, required for class visualization graph**
    10. derived.isastar.tsv.gz **# This file is required for creating the ES index only.**

The files `metadata.pagerank.undirected.tsv.gz` and `derived.isastar.tsv.gz` are created by running [this](https://github.com/usc-isi-i2/kgtk/blob/dev/use-cases/Wikidata%20Useful%20Files.ipynb) notebook
- output_path: Output path 
- project_name: folder inside the `output_path` where the required files and cache will be created
- es_host: ES host, default `localhost`
- es_port: ES port, default `9200`
- es_index: name of the desired elasticsearch index, default `wikidata-dwd-kgtk-search-01`
- create_db: variable to control creation of sqlite database cache. `yes|no`, default `yes`
- create_es: variable to control creation of the ES index. `yes|no`, defaut `no`
- create_class_viz: variable to control creation of class visualization tables in the sqlite database cache. Most probably, if you are not working with Wikidata, you will not have files # 8 and 9 listed above. Set this parameter to 'no' in that case. `yes|no`, default `yes`

**Cache file location:** `<output_path>/<project_name>/temp.<project_name>/wikidata.sqlite3.db`

In [1]:
import os
import pandas as pd
from kgtk.configure_kgtk_notebooks import ConfigureKGTK
import kgtk.kypher.api as kapi
from kgtk.functions import kgtk, kypher

In [2]:
input_path = "/data02/ana_iglesias/data/subset/reframings/events"
output_path = "/data02/ana_iglesias/data/subset"

project_name = "KRHC-NAry-Events"

files = 'label,pagerank_undirected,alias,description,claims,datatypes,qualifiers'

es_host = "http://localhost"
es_port = 9200
es_index = "wikidata-dwd-kgtk-search-01"

create_db = 'yes'
create_es = 'no'
create_class_viz = 'no'

In [3]:
files = files.split(',')

create_sqlite_cache = create_db.lower() == 'yes'
create_es_index = create_es.lower() == 'yes'
create_class_viz_tables = create_class_viz.lower() == 'yes'

additional_files = {
    'classvizedge': 'class-visualization.edge.tsv.gz',
    'classviznode': 'class-visualization.node.tsv.gz'
}

In [4]:
ck = ConfigureKGTK(files)
if create_class_viz_tables:
    ck.configure_kgtk(input_graph_path=input_path,
                      output_path=output_path,
                      project_name=project_name,
                     additional_files=additional_files)
else:
    ck.configure_kgtk(input_graph_path=input_path,
                      output_path=output_path,
                      project_name=project_name)

User home: /data02/ana_iglesias
Current dir: /data02/ana_iglesias/data/subset
KGTK dir: /data02/ana_iglesias/data
Use-cases dir: /data02/ana_iglesias/data/use-cases


In [5]:
ck.print_env_variables()

kypher: kgtk query --graph-cache /data02/ana_iglesias/data/subset/KRHC-NAry-Events/temp.KRHC-NAry-Events/wikidata.sqlite3.db
GRAPH: /data02/ana_iglesias/data/subset/reframings/events
TEMP: /data02/ana_iglesias/data/subset/KRHC-NAry-Events/temp.KRHC-NAry-Events
STORE: /data02/ana_iglesias/data/subset/KRHC-NAry-Events/temp.KRHC-NAry-Events/wikidata.sqlite3.db
KGTK_OPTION_DEBUG: false
KGTK_GRAPH_CACHE: /data02/ana_iglesias/data/subset/KRHC-NAry-Events/temp.KRHC-NAry-Events/wikidata.sqlite3.db
OUT: /data02/ana_iglesias/data/subset/KRHC-NAry-Events
USE_CASES_DIR: /data02/ana_iglesias/data/use-cases
EXAMPLES_DIR: /data02/ana_iglesias/data/examples
kgtk: kgtk
KGTK_LABEL_FILE: /data02/ana_iglesias/data/subset/reframings/events/labels.en.tsv.gz
label: /data02/ana_iglesias/data/subset/reframings/events/labels.en.tsv.gz
pagerank_undirected: /data02/ana_iglesias/data/subset/reframings/events/metadata.pagerank.undirected.tsv.gz
alias: /data02/ana_iglesias/data/subset/reframings/events/aliases.en.ts

## Load the files into cache

In [6]:
if create_sqlite_cache:
    ck.load_files_into_cache()

kgtk query --graph-cache /data02/ana_iglesias/data/subset/KRHC-NAry-Events/temp.KRHC-NAry-Events/wikidata.sqlite3.db -i "/data02/ana_iglesias/data/subset/reframings/events/labels.en.tsv.gz" --as label  -i "/data02/ana_iglesias/data/subset/reframings/events/metadata.pagerank.undirected.tsv.gz" --as pagerank_undirected  -i "/data02/ana_iglesias/data/subset/reframings/events/aliases.en.tsv.gz" --as alias  -i "/data02/ana_iglesias/data/subset/reframings/events/descriptions.en.tsv.gz" --as description  -i "/data02/ana_iglesias/data/subset/reframings/events/claims.tsv.gz" --as claims  -i "/data02/ana_iglesias/data/subset/reframings/events/metadata.property.datatypes.tsv.gz" --as datatypes  -i "/data02/ana_iglesias/data/subset/reframings/events/qualifiers.tsv.gz" --as qualifiers  --limit 3
node1	label	node2	id
P10	label	'video'@en	P10-label-en
P1000	label	'record held'@en	P1000-label-en
P10000	label	'Research Vocabularies Australia ID'@en	P10000-label-en


## Define the Kypher API

In [7]:
_kapi2 = kapi.KypherApi(graphcache=os.environ['STORE'], loglevel=1, index='auto',
                      maxresults=100, maxcache=0)

## Create a file with `label`, `undirected_pagerank` and `description`

In [8]:
if create_sqlite_cache:
    !kgtk query --gc $STORE \
        -i label pagerank_undirected description\
        --match 'label: (qnode)-[l]->(y), pagerank: (qnode)-[:Pundirected_pagerank]->(pr)' \
        --opt 'description: (qnode)-[:description]->(d)' \
        --return 'qnode as node1, l.label as label, y as node2, upper(y) as `node2;upper`, pr as `node1;pagerank`, ifnull(d, "") as `node1;description`' \
        --order-by 'qnode' \
        -o $OUT/label_pagerank_undirected_description.tsv.gz

### Load this file into cache as well

In [9]:
if create_sqlite_cache:
    !kgtk query --gc $STORE -i $OUT/label_pagerank_undirected_description.tsv.gz --as l_d_pgr_ud --limit 10

node1	label	node2	node2;upper	node1;pagerank	node1;description
P10	label	'video'@en	'VIDEO'@EN	1.90092695780294e-06	'relevant video. For images, use the property P18. For film trailers, qualify with \"object has role\" (P3831)=\"trailer\" (Q622550)'@en
P1000	label	'record held'@en	'RECORD HELD'@EN	8.846157108863055e-07	'notable record achieved by a person or entity, include qualifiers for dates held'@en
P10000	label	'Research Vocabularies Australia ID'@en	'RESEARCH VOCABULARIES AUSTRALIA ID'@EN	1.7379313655369044e-06	'identifier of a vocabulary in Research Vocabularies Australia'@en
P10006	label	'AllSides ID'@en	'ALLSIDES ID'@EN	1.8528701572707561e-06	'alphabetical identifier of a publication, organization, or person at AllSides'@en
P10007	label	'Birdata ID'@en	'BIRDATA ID'@EN	1.275801866591085e-06	'identifier for an bird species in the Birdata database'@en
P10008	label	'Geographical Names Board of NSW geoname ID'@en	'GEOGRAPHICAL NAMES BOARD OF NSW GEONAME ID'@EN	2.0272204848959872e-0

## Create the required indices

In [10]:
%%time 
if create_sqlite_cache:
    !kgtk --debug query -i l_d_pgr_ud --idx node1 "node2;upper" label text:node2//name=ldpgridx --gc $STORE --limit 5

[2022-10-03 18:56:01 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_8 AS graph_8_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-10-03 18:56:01 sqlstore]: CREATE INDEX "graph_8_node1_idx" ON "graph_8" ("node1")
[2022-10-03 18:56:02 sqlstore]: ANALYZE "graph_8_node1_idx"
[2022-10-03 18:56:02 sqlstore]: CREATE INDEX "graph_8_node2;upper_idx" ON "graph_8" ("node2;upper")
[2022-10-03 18:56:03 sqlstore]: ANALYZE "graph_8_node2;upper_idx"
[2022-10-03 18:56:03 sqlstore]: CREATE INDEX "graph_8_label_idx" ON "graph_8" ("label")
[2022-10-03 18:56:03 sqlstore]: ANALYZE "graph_8_label_idx"
[2022-10-03 18:56:04 sqlstore]: CREATE VIRTUAL TABLE "graph_8_txtidx_ldpgridx" USING FTS5 ("node2", tokenize="trigram", content="graph_8")
[2022-10-03 18:56:04 sqlstore]: INSERT INTO "graph_8_txtidx_ldpgridx" ("node2") SELECT "node2" FROM graph_8
node1	label	node2	node2;upper	node1;pagerank	node1;description
P10	label	'video'@en	

In [11]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i label --idx label --gc $STORE --limit 5

[2022-10-03 18:56:11 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_1 AS graph_1_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-10-03 18:56:11 sqlstore]: CREATE INDEX "graph_1_label_idx" ON "graph_1" ("label")
[2022-10-03 18:56:11 sqlstore]: ANALYZE "graph_1_label_idx"
node1	label	node2	id
P10	label	'video'@en	P10-label-en
P1000	label	'record held'@en	P1000-label-en
P10000	label	'Research Vocabularies Australia ID'@en	P10000-label-en
P10006	label	'AllSides ID'@en	P10006-label-en
P10007	label	'Birdata ID'@en	P10007-label-en
CPU times: user 25 ms, sys: 14.2 ms, total: 39.2 ms
Wall time: 1.67 s


In [12]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i alias --idx label --gc $STORE --limit 5

[2022-10-03 18:56:13 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_3 AS graph_3_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-10-03 18:56:13 sqlstore]: CREATE INDEX "graph_3_label_idx" ON "graph_3" ("label")
[2022-10-03 18:56:13 sqlstore]: ANALYZE "graph_3_label_idx"
node1	label	node2	id
P10	alias	'animation'@en	P10-alias-en-2f86d8-0
P10	alias	'gif'@en	P10-alias-en-282226-0
P10	alias	'media'@en	P10-alias-en-c1427e-0
P10	alias	'trailer (Commons)'@en	P10-alias-en-c61ab1-0
P10000	alias	'Australian Research Vocabularies ID'@en	P10000-alias-en-0df7f5-0
CPU times: user 28.4 ms, sys: 17.4 ms, total: 45.8 ms
Wall time: 1.72 s


In [13]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i description --idx id --gc $STORE --limit 5

[2022-10-03 18:56:14 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_4 AS graph_4_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-10-03 18:56:14 sqlstore]: CREATE INDEX "graph_4_id_idx" ON "graph_4" ("id")
[2022-10-03 18:56:15 sqlstore]: ANALYZE "graph_4_id_idx"
node1	label	node2	id
P10	description	'relevant video. For images, use the property P18. For film trailers, qualify with \"object has role\" (P3831)=\"trailer\" (Q622550)'@en	P10-description-en
P1000	description	'notable record achieved by a person or entity, include qualifiers for dates held'@en	P1000-description-en
P10000	description	'identifier of a vocabulary in Research Vocabularies Australia'@en	P10000-description-en
P10006	description	'alphabetical identifier of a publication, organization, or person at AllSides'@en	P10006-description-en
P10007	description	'identifier for an bird species in the Birdata database'@en	P10007-description-en
CPU

In [14]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i claims --idx label node1 node2 id --gc $STORE --limit 5

[2022-10-03 18:56:16 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_5 AS graph_5_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-10-03 18:56:16 sqlstore]: CREATE INDEX "graph_5_label_idx" ON "graph_5" ("label")
[2022-10-03 18:56:21 sqlstore]: ANALYZE "graph_5_label_idx"
[2022-10-03 18:56:22 sqlstore]: CREATE INDEX "graph_5_node1_idx" ON "graph_5" ("node1")
[2022-10-03 18:56:24 sqlstore]: ANALYZE "graph_5_node1_idx"
[2022-10-03 18:56:25 sqlstore]: CREATE INDEX "graph_5_node2_idx" ON "graph_5" ("node2")
[2022-10-03 18:56:32 sqlstore]: ANALYZE "graph_5_node2_idx"
[2022-10-03 18:56:32 sqlstore]: CREATE INDEX "graph_5_id_idx" ON "graph_5" ("id")
[2022-10-03 18:56:37 sqlstore]: ANALYZE "graph_5_id_idx"
node1	label	node2	id
P10	P1628	"http://www.w3.org/2006/vcard/ns#Video"	P10-P1628-32b85d-7927ece6-0
P10	P1628	"https://schema.org/video"	P10-P1628-acf60d-b8950832-0
P10	P1629	Q34508	P10-P1629-Q34508-bcc39400-0
P

In [15]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i datatypes --idx label node1 --gc $STORE --limit 5

[2022-10-03 18:56:40 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_6 AS graph_6_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-10-03 18:56:40 sqlstore]: CREATE INDEX "graph_6_label_idx" ON "graph_6" ("label")
[2022-10-03 18:56:40 sqlstore]: ANALYZE "graph_6_label_idx"
[2022-10-03 18:56:40 sqlstore]: CREATE INDEX "graph_6_node1_idx" ON "graph_6" ("node1")
[2022-10-03 18:56:40 sqlstore]: ANALYZE "graph_6_node1_idx"
node1	label	node2	id
Psubject	datatype	wikibase-item	Psubject-datatype
Ppredicate	datatype	wikibase-item	Ppredicate-datatype
Pobject	datatype	string	Pobject-datatype
Phas_event	datatype	wikibase-item	Pevent-datatype
P10	datatype	commonsMedia	P10-datatype
CPU times: user 24.1 ms, sys: 9.25 ms, total: 33.4 ms
Wall time: 1.38 s


In [16]:
%%time
if create_sqlite_cache:
    !kgtk --debug query -i qualifiers --idx node2 node1 label --gc $STORE --limit 5

[2022-10-03 18:56:41 query]: SQL Translation:
---------------------------------------------
  SELECT *
     FROM graph_7 AS graph_7_c1
     LIMIT ?
  PARAS: [5]
---------------------------------------------
[2022-10-03 18:56:41 sqlstore]: CREATE INDEX "graph_7_node2_idx" ON "graph_7" ("node2")
[2022-10-03 18:56:41 sqlstore]: ANALYZE "graph_7_node2_idx"
[2022-10-03 18:56:41 sqlstore]: CREATE INDEX "graph_7_node1_idx" ON "graph_7" ("node1")
[2022-10-03 18:56:41 sqlstore]: ANALYZE "graph_7_node1_idx"
[2022-10-03 18:56:41 sqlstore]: CREATE INDEX "graph_7_label_idx" ON "graph_7" ("label")
[2022-10-03 18:56:42 sqlstore]: ANALYZE "graph_7_label_idx"
node1	label	node2	id
CPU times: user 20.4 ms, sys: 17.1 ms, total: 37.5 ms
Wall time: 1.51 s
